# Exploratory data analysis

Useful resources:
 - https://colab.research.google.com/github/mdai/ml-lessons/blob/master/lesson2-lung-xrays-segmentation.ipynb
 - https://github.com/ayush1997/Xvision
 - https://www.kaggle.com/zahaviguy/what-are-lung-opacities
 - https://www.kaggle.com/giuliasavorgnan/data-leakage-viewposition-pa-ap-matters

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os
import sys
import tqdm
import warnings
import logging
from pathlib import Path
import numpy as np

warnings.filterwarnings("ignore")
sys.path.append(str(Path(os.getcwd() + '/../../').resolve()))
np.warnings.filterwarnings('ignore')

In [2]:
import pydicom
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import Image, display
import tensorflow as tf

from rsna.data import Dataset
from rsna import main

## Get a feel of the dataset

In [3]:
logger = main.getLogger()
input_dir = (Path(os.getcwd()) / ".." / ".." / "input" / "rsna-pneumonia-detection-challenge").resolve()

ds = Dataset(logger, input_dir)
df, lung_class_ltable = ds.read_training_data()
df.head()

,patientId,x,y,width,height,Target,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,-1,-1,-1,-1,0,1
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,-1,-1,-1,-1,0,1
2,00322d4d-1c29-4943-afc9-b6754be640eb,-1,-1,-1,-1,0,1
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,-1,-1,-1,-1,0,2
4,00436515-870c-4b36-a041-de91049b9ab4,264,152,213,379,1,0


## Are there pneumonia images in the No Lung Opacity / Not Normal class?

In [4]:
df[["Target", "class"]].corr()

,Target,class
Target,1.000000,-0.886577
class,-0.886577,1.000000
